In [139]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from collections import defaultdict
import shutil
from itertools import chain

# Проверка описания товара

In [8]:
desc = pd.read_csv("C:/temp/!hse-mlds-project-year2/data/Ювелирные&изделия_Четки/descriptions.csv")

In [9]:
desc

,item,description
0,0.png,ВНИМАНИЕ! Украшение для категории аксессуары в...
1,1.png,Четки Православные изготовлены из Кахолонга д-...
2,2.png,Четки Мусульманские изготовлены из зеленого ав...
3,3.png,"На бирке вес , штрих код и УИН , по которому В..."
4,4.png,Мусульманские четки изготовлены Агата д-10 мм ...
5,5.png,NaN
6,6.png,Мусульманские четки изготовлены из Кахолонга д...
7,7.png,Четки Мусульманские из агата 10 мм со вставкам...
8,8.png,Ювелирные мусульманские религиозные четки Куб...
9,9.png,Четки мусульманские изготовлены из граненого ч...


____

# Сбор статистики по спаршенным фотографиям

In [10]:
root_dir = Path("C:/temp/!hse-mlds-project-year2/woman/woman")

In [11]:
# Кол-во категорий
folders = os.listdir(root_dir)
len(folders)

13

In [15]:
df_dict = {col: [] for col in ["card", "feedbacks"]}

for cat in folders:
    path = root_dir / cat
    path_card = path / "card"
    path_feedbacks = path / "feedbacks"

    # кол-во загруженных фоток
    load_card = len(sorted(path_card.glob('*')))
    load_feedbacks = len(sorted(path_feedbacks.glob('*')))
    
    if load_feedbacks < 20:
        print(f"{load_feedbacks} Feedbacks in {cat}")

#     if load_card < 20:
#         print(f"{load_card} Card in {cat}")

    df_dict["card"].append(load_card)
    df_dict["feedbacks"].append(load_feedbacks)

19 Feedbacks in Женщинам_Толстовки,&свитшоты&и&худи
19 Feedbacks in Женщинам_Туники


In [16]:
df = pd.DataFrame(df_dict)
df.head()

,card,feedbacks
0,20,20
1,20,20
2,20,20
3,20,20
4,20,20


In [17]:
df.describe()

,card,feedbacks
count,13.0,13.000000
mean,20.0,19.846154
std,0.0,0.375534
min,20.0,19.000000
25%,20.0,20.000000
50%,20.0,20.000000
75%,20.0,20.000000
max,20.0,20.000000


In [18]:
df.feedbacks.value_counts()

feedbacks
20    11
19     2
Name: count, dtype: int64

In [8]:
df.card.value_counts()

card
20    1575
19       2
18       1
4        1
1        1
Name: count, dtype: int64

18 Card in Дом_Кухня_Кухонный&текстиль_Чехлы&для&ручек&холодильников  
4 Card in Дом_Освещение_Лифты&для&люстр                                _(удалить либо перенести)_  
19 Card in Мебель_Гардеробная&мебель_Ящики                             _(можно до 20 добрать)_  
1 Card in Мебель_Офисная&мебель_Перегородки&офисные                    _(удалить либо перенести)_  
19 Card in Мебель_Офисная&мебель_Шкафы  

___

# Проверка на дубликаты ссылок

In [180]:
categories = pd.read_csv("wb_categories_full_clear.csv")
categories.head()

,category_1,category_2,category_3,category_4,category_5,url
0,Автотовары,Мойки высокого давления и аксессуары,Цепные пилы,NaN,NaN,https://www.wildberries.ru/catalog/dom/dachniy...
1,Автотовары,Автокосметика и автохимия,Присадки,NaN,NaN,https://www.wildberries.ru/catalog/avtotovary/...
2,Автотовары,Автокосметика и автохимия,Смазки автомобильные,NaN,NaN,https://www.wildberries.ru/catalog/avtotovary/...
3,Автотовары,Автокосметика и автохимия,Уход за стеклами и фарами,NaN,NaN,https://www.wildberries.ru/catalog/aksessuary/...
4,Автотовары,Автокосметика и автохимия,Уход за салоном,NaN,NaN,https://www.wildberries.ru/catalog/aksessuary/...


In [181]:
df_dupl = categories[categories.duplicated(['url'], keep=False)]
df_dupl

,category_1,category_2,category_3,category_4,category_5,url
0,Автотовары,Мойки высокого давления и аксессуары,Цепные пилы,NaN,NaN,https://www.wildberries.ru/catalog/dom/dachniy...
13,Автотовары,Автоэлектроника и навигация,NaN,NaN,NaN,https://www.wildberries.ru/catalog/elektronika...
14,Автотовары,Аккумуляторы и сопутствующие товары,NaN,NaN,NaN,https://www.wildberries.ru/catalog/aksessuary/...
48,Автотовары,Мойки высокого давления и аксессуары,Аксессуары и запчасти для садовой техники,NaN,NaN,https://www.wildberries.ru/catalog/dom/dachniy...
49,Автотовары,Мойки высокого давления и аксессуары,Садовые измельчители,NaN,NaN,https://www.wildberries.ru/catalog/dom/dachniy...
...,...,...,...,...,...,...
1647,Электроника,Смартфоны и телефоны,Стационарные телефоны,NaN,NaN,https://www.wildberries.ru/catalog/elektronika...
1648,Электроника,Смартфоны и телефоны,Смартфоны,NaN,NaN,https://www.wildberries.ru/catalog/elektronika...
1649,Электроника,Смартфоны и телефоны,Планшеты,NaN,NaN,https://www.wildberries.ru/catalog/elektronika...
1650,Электроника,Смартфоны и телефоны,Мобильные телефоны,NaN,NaN,https://www.wildberries.ru/catalog/elektronika...


In [182]:
df_dupl.index

Index([   0,   13,   14,   48,   49,   50,   51,   52,   68,   69,
       ...
       1640, 1641, 1642, 1643, 1646, 1647, 1648, 1649, 1650, 1651],
      dtype='int64', length=156)

In [183]:
duplicates = defaultdict(list)

for ind in df_dupl.index:
    url = df_dupl.loc[ind]["url"]
    duplicates[url].append(ind)

In [184]:
print(len(duplicates))
duplicates

69


defaultdict(list,
            {'https://www.wildberries.ru/catalog/dom/dachniy-sezon/sadovaya-tehnika/tsepnye-pily': [0,
              1440,
              1480],
             'https://www.wildberries.ru/catalog/elektronika/avtoelektronika': [13,
              259],
             'https://www.wildberries.ru/catalog/aksessuary/avtotovary/akkumulyatory-i-soputstvuyushchie-tovary': [14,
              1588],
             'https://www.wildberries.ru/catalog/dom/dachniy-sezon/sadovaya-tehnika/aksessuary-i-zapchasti-dlya-sadovoy-tehniki': [48,
              1473,
              1475],
             'https://www.wildberries.ru/catalog/dom/dachniy-sezon/sadovaya-tehnika/sadovye-izmelchiteli': [49,
              1470,
              1476],
             'https://www.wildberries.ru/catalog/dom/dachniy-sezon/sadovaya-tehnika/motobury-i-aksessuary-k-nim': [50,
              1478,
              1498],
             'https://www.wildberries.ru/catalog/dom/dachniy-sezon/sadovaya-tehnika/kultivatory-i-motoblo

In [185]:
categories["url"][207], categories["url"][976]

('https://www.wildberries.ru/catalog/podarki/detyam/shariki-i-upakovka',
 'https://www.wildberries.ru/catalog/knigi/bukinistika/lyubovnye-romany')

In [186]:
df_duplicates = pd.DataFrame(duplicates.items(), columns=["url", "index"])

In [187]:
#df_duplicates.to_csv("wb_categories_dublicates.csv", index=False)

In [24]:
df = pd.DataFrame({
    'brand': ['Yum Yum', 'Yum Yum', 'Indomie', 'Indomie', 'Indomie'],
    'style': ['cup', 'cup', 'cup', 'pack', 'pack'],
    'rating': [4, 4, 3.5, 15, 5]
})
df

,brand,style,rating
0,Yum Yum,cup,4.0
1,Yum Yum,cup,4.0
2,Indomie,cup,3.5
3,Indomie,pack,15.0
4,Indomie,pack,5.0


In [25]:
df[df.duplicated(['style'], keep=False)]

,brand,style,rating
0,Yum Yum,cup,4.0
1,Yum Yum,cup,4.0
2,Indomie,cup,3.5
3,Indomie,pack,15.0
4,Indomie,pack,5.0


___

## удаление дубликатов

In [188]:
df_duplicates_choose = pd.read_csv("E://!docum//!HSE магистратура//Задание диплом//wb_categories_dublicates.csv")
#df_duplicates = pd.read_csv("wb_categories_dublicates.csv")

df_duplicates_choose = pd.concat([df_duplicates_choose[:12],
                                  pd.DataFrame([np.nan], columns=["choose_category"]),
                                  df_duplicates_choose[12:]], axis=0)
df_duplicates_choose.reset_index(drop=True, inplace=True)

df_duplicates = pd.concat([df_duplicates, df_duplicates_choose], axis=1)
#df_duplicates["delete"] = df_duplicates["index"].copy(deep=True)

In [189]:
df_duplicates.apply(lambda x : x["index"].remove(x["choose_category"]) if (str(x["choose_category"]) != "nan") else x["index"], axis=1);

In [190]:
df_duplicates

,url,index,choose_category
0,https://www.wildberries.ru/catalog/dom/dachniy...,"[0, 1440]",1480.0
1,https://www.wildberries.ru/catalog/elektronika...,[259],13.0
2,https://www.wildberries.ru/catalog/aksessuary/...,[1588],14.0
3,https://www.wildberries.ru/catalog/dom/dachniy...,"[48, 1473]",1475.0
4,https://www.wildberries.ru/catalog/dom/dachniy...,"[49, 1470]",1476.0
...,...,...,...
64,https://www.wildberries.ru/catalog/elektronika...,[1537],1642.0
65,https://www.wildberries.ru/catalog/elektronika...,[1549],1634.0
66,https://www.wildberries.ru/catalog/elektronika...,[1558],1633.0
67,https://www.wildberries.ru/catalog/elektronika...,[1559],1632.0


In [200]:
root_dir = Path("C:/temp/!hse-mlds-project-year2/data")

In [201]:
folders_name = []

for i in range(len(categories)):
    if str(categories.iloc[i]["category_5"]) == "nan":
        if str(categories.iloc[i]["category_4"]) == "nan":
            if str(categories.iloc[i]["category_3"]) == "nan":
                if str(categories.iloc[i]["category_2"]) == "nan":
                    cat = list(categories.iloc[i][:1])
                else:
                    cat = list(categories.iloc[i][:2])
            else:
                cat = list(categories.iloc[i][:3])
        else:
            cat = list(categories.iloc[i][:4])
    else:
        cat = list(categories.iloc[i][:5])
    folder = "_".join(["&".join(i.split()) for i in cat])
    folders_name.append(folder)

In [209]:
delete_index = list(chain(*list(df_duplicates["index"])))

In [214]:
len(delete_index)

88

In [211]:
for ind in delete_index:
    shutil.rmtree(root_dir / folders_name[ind])

___

# Выбор категории

In [3]:
root_dir = Path("C:/temp/!hse-mlds-project-year2/data")

In [4]:
# Кол-во категорий
folders = os.listdir(root_dir)
len(folders)

1580

In [5]:
def choose_category(level, folders, category1=None, category2=None, category3=None, category4=None):
    if level == 1:
        tree = {(category.split("_")[0], 2) if len(category.split("_"))>1 else (category.split("_")[0], 1) for category in folders}

    elif level == 2:
        if category1 is None:
            raise TypeError("choose_category() missing 1 required positional argument: 'category1'")
        categories = [category for category in folders if category.split("_")[0]==category1]
        tree = {(category.split("_")[1], 3) if len(category.split("_"))>2 else (category.split("_")[1], 2) for category in categories}

    elif level == 3:
        if category1 is None or category2 is None:
            raise TypeError("choose_category() missing required positional argument: 'category1' or 'category2'")
        categories = [category for category in folders if (category.split("_")[0]==category1) and (category.split("_")[1]==category2)]
        tree = {(category.split("_")[2], 4) if len(category.split("_"))>3 else (category.split("_")[2], 3) for category in categories}

    elif level == 4:
        if category1 is None or category2 is None or category3 is None:
            raise TypeError("choose_category() missing required positional argument: 'category1', 'category2' or 'category3'")
        categories = [category for category in folders if (category.split("_")[0]==category1) and (category.split("_")[1]==category2) and (category.split("_")[2]==category3)]
        tree = {(category.split("_")[3], 5) if len(category.split("_"))>4 else (category.split("_")[3], 4) for category in categories}

    elif level == 5:
        if category1 is None or category2 is None or category3 is None  or category4 is None:
            raise TypeError("choose_category() missing required positional argument: 'category1', 'category2', 'category3' or 'category4'")
        categories = [category for category in folders if (category.split("_")[0]==category1) and (category.split("_")[1]==category2) and (category.split("_")[2]==category3) and (category.split("_")[3]==category4)]
        tree = {(category.split("_")[4], 6) if len(category.split("_"))>5 else (category.split("_")[4], 5) for category in categories}
    
    return sorted(tree)

In [81]:
choose_category(5, folders, "Дом", "Предметы&интерьера", 'Картины&и&постеры', 'Картины')

[('Аниме', 5),
 ('Арт&и&абстракция', 5),
 ('Архитектура', 5),
 ('Гобелены', 5),
 ('Города&и&страны', 5),
 ('Детские', 5),
 ('Еда&и&напитки', 5),
 ('Животные&и&птицы', 5),
 ('Интерьеры', 5),
 ('Карты', 5),
 ('Кино&и&музыка', 5),
 ('Коллажи', 5),
 ('Комиксы', 5),
 ('Компьютерные&игры', 5),
 ('Космос', 5),
 ('Люди', 5),
 ('Надписи', 5),
 ('Натюрморты', 5),
 ('Панно', 5),
 ('Праздники', 5),
 ('Природа', 5),
 ('Религия', 5),
 ('Репродукции', 5),
 ('Ретро', 5),
 ('Спорт', 5),
 ('Транспорт', 5),
 ('Фотография', 5),
 ('Фэнтези', 5),
 ('Цветы&и&растения', 5),
 ('Юмор', 5)]

In [34]:
choose_category(4, folders, "Дом", "Предметы&интерьера", 'Картины&и&постеры', 'Картины')

[('Гравюры', 4),
 ('Доски&для&записей', 4),
 ('Картины', 5),
 ('Картины&3D', 4),
 ('Картины&в&рамах', 4),
 ('Модульные&картины', 4),
 ('Постеры', 5),
 ('Постеры&в&рамах', 4),
 ('Рамы&багетные', 4),
 ('Рамы&для&постеров', 4),
 ('Таблички&информационные', 4)]

___

# Расчет кол-ва моделей в древовидном инференсе

In [43]:
num_models = 1
categories1 = {(category.split("_")[0], 2) for category in folders if len(category.split("_"))>1}

for cat1 in categories1:
    num_models += 1
    categories = [category for category in folders if category.split("_")[0]==cat1[0]]
    categories2 = {(category.split("_")[1], 3) for category in categories if len(category.split("_"))>2}
    
    for cat2 in categories2:
        num_models += 1
        categories = [category for category in folders if (category.split("_")[0]==cat1[0]) and (category.split("_")[1]==cat2[0])]
        categories3 = {(category.split("_")[2], 4) for category in categories if len(category.split("_"))>3}
        
        for cat3 in categories3:
            num_models += 1
            categories = [category for category in folders if (category.split("_")[0]==cat1[0]) and (category.split("_")[1]==cat2[0]) and (category.split("_")[2]==cat3[0])]
            categories4 = {(category.split("_")[3], 5) for category in categories if len(category.split("_"))>4}
            
            for cat4 in categories4:
                num_models += 1

In [44]:
num_models

171

Кол-во моделей:
 - `171 num_models` вложенность 5
 - `165 num_models` вложенность 4
 - `153 num_models` вложенность 3
 - `22 num_models` вложенность 2
 - `1 num_models` вложенность 1

___

# Функции для обработки датасета

In [179]:
root_dir = Path("C:/temp/!hse-mlds-project-year2/test")

In [180]:
# Кол-во категорий
folders = os.listdir(root_dir)
print(len(folders))
folders

3


['Ювелирные&изделия_Украшения&из&серебра',
 'Ювелирные&изделия_Часы',
 'Ювелирные&изделия_Четки']

## удаление товаров

удаляет только feedback для указанного товара

In [16]:
def delete_feedbacks(path: Path, ind: list[int]):
    path = path / "feedbacks"
    paths_to_img = [path / f"{i}_feedbacks.png" for i in ind]
    for p in paths_to_img:
        os.remove(p)
    
    return paths_to_img

In [19]:
delete_feedbacks(path=root_dir / folders[2], ind=[4,6])

[WindowsPath('C:/temp/!hse-mlds-project-year2/test/Ювелирные&изделия_Четки/feedbacks/4_feedbacks.png'),
 WindowsPath('C:/temp/!hse-mlds-project-year2/test/Ювелирные&изделия_Четки/feedbacks/6_feedbacks.png')]

удаляет полностью card, description и feedback (если он есть) для указанного товара

In [183]:
def delete_card_and_description(path: Path, ind: list[int]):
    path_card = path / "card"
    path_feedbacks = path / "feedbacks"
    paths_card_img = [path_card / f"{i}.png" for i in ind]
    paths_feedbacks_img = [path_feedbacks / f"{i}_feedbacks.png" for i in ind]
    check_feedbacks = [True if path.exists() else False for path in paths_feedbacks_img]

    descriptions =  pd.read_csv(path / "descriptions.csv")
    descriptions.set_index(['item'], inplace=True)
    for i in range(len(ind)):
        
        # удаление описания
        try:
            descriptions.drop(index=paths_card_img[i].parts[-1], inplace=True)
        except Exception as e:
            print(e)
            return f"Упало на {paths_card_img[i]}"
        
        # удаление card
        try:
            os.remove(paths_card_img[i])
        except Exception as e:
            print(e)
            return f"Упало на {paths_card_img[i]}"
        
        # удаление feedbacks
        if check_feedbacks[i]:
            try:
                os.remove(paths_feedbacks_img[i])
            except Exception as e:
                print(e)
                return f"Упало на {paths_feedbacks_img[i]}"
        
    descriptions.reset_index(inplace=True)
    descriptions.to_csv(path / "descriptions.csv", index=False)
    return paths_card_img

In [188]:
paths_to_img = delete_card_and_description(path=root_dir / folders[0], ind=[9,10,12])
paths_to_img

[WindowsPath('C:/temp/!hse-mlds-project-year2/test/Ювелирные&изделия_Украшения&из&серебра/card/9.png'),
 WindowsPath('C:/temp/!hse-mlds-project-year2/test/Ювелирные&изделия_Украшения&из&серебра/card/10.png'),
 WindowsPath('C:/temp/!hse-mlds-project-year2/test/Ювелирные&изделия_Украшения&из&серебра/card/12.png')]

## перенос товара в другую категорию

переносится полностью card, description и feedback (если он есть) для указанного товара

In [191]:
def move_items_to_another_category(path_from: Path,
                                   path_to: Path,
                                   ind: list[int]):
    from_path_card = path_from / "card"
    from_path_feedbacks = path_from / "feedbacks"
    from_paths_card_img = [from_path_card / f"{i}.png" for i in ind]
    from_paths_feedbacks_img = [from_path_feedbacks / f"{i}_feedbacks.png" for i in ind]
    check_feedbacks = [True if path.exists() else False for path in from_paths_feedbacks_img]

    to_path_card = path_to / "card"
    to_path_feedbacks = path_to / "feedbacks"
    start_name = int(sorted(path.rglob("*.png"), key=lambda x: int(x.stem.split('_')[0]))[-1].stem.split('_')[0])
    to_paths_card_img = [to_path_card / f"{i}.png" for i in range(start_name+1, start_name+1+len(ind))]
    to_paths_feedbacks_img = [to_path_feedbacks / f"{i}_feedbacks.png" for i in range(start_name+1, start_name+1+len(ind))]

    from_descriptions = pd.read_csv(path_from / "descriptions.csv")
    from_descriptions.set_index(['item'], inplace=True)
    to_descriptions = pd.read_csv(path_to / "descriptions.csv")
    to_descriptions.set_index(['item'], inplace=True)
    
    for i in range(len(ind)):
        
        # перемещение описания
        try:
            to_descriptions.loc[to_paths_card_img[i].parts[-1]] = from_descriptions.loc[from_paths_card_img[i].parts[-1]]["description"]
            from_descriptions.drop(index=from_paths_card_img[i].parts[-1], inplace=True)
        except Exception as e:
            print(e)
            return f"Упало на description of {from_paths_card_img[i]}"
        
        # перемещение card
        try:
            shutil.move(from_paths_card_img[i], to_paths_card_img[i])
        except Exception as e:
            print(e)
            return f"Упало на {from_paths_card_img[i]}"
        
        # перемещение feedbacks
        if check_feedbacks[i]:
            try:
                shutil.move(from_paths_feedbacks_img[i], to_paths_feedbacks_img[i])
            except Exception as e:
                print(e)
                return f"Упало на {from_paths_feedbacks_img[i]}"
            
    from_descriptions.reset_index(inplace=True)
    to_descriptions.reset_index(inplace=True)
    from_descriptions.to_csv(path_from / "descriptions.csv", index=False)
    to_descriptions.to_csv(path_to / "descriptions.csv", index=False)

    return (from_paths_card_img, to_paths_card_img)

In [192]:
logs = move_items_to_another_category(path_from=root_dir / folders[2],
                                      path_to=root_dir / folders[1],
                                      ind=[1,4,6])
logs

([WindowsPath('C:/temp/!hse-mlds-project-year2/test/Ювелирные&изделия_Четки/card/1.png'),
  WindowsPath('C:/temp/!hse-mlds-project-year2/test/Ювелирные&изделия_Четки/card/4.png'),
  WindowsPath('C:/temp/!hse-mlds-project-year2/test/Ювелирные&изделия_Четки/card/6.png')],
 [WindowsPath('C:/temp/!hse-mlds-project-year2/test/Ювелирные&изделия_Часы/card/20.png'),
  WindowsPath('C:/temp/!hse-mlds-project-year2/test/Ювелирные&изделия_Часы/card/21.png'),
  WindowsPath('C:/temp/!hse-mlds-project-year2/test/Ювелирные&изделия_Часы/card/22.png')])

проверка описания

In [195]:
path = root_dir / folders[2]
descriptions = pd.read_csv(path / "descriptions.csv")
descriptions.set_index(['item'], inplace=True)

In [193]:
path = root_dir / folders[1]
descriptions2 = pd.read_csv(path / "descriptions.csv")
descriptions2.set_index(['item'], inplace=True)

In [196]:
descriptions

,description
item,
0.png,ВНИМАНИЕ! Украшение для категории аксессуары в...
2.png,Четки Мусульманские изготовлены из зеленого ав...
3.png,"На бирке вес , штрих код и УИН , по которому В..."
5.png,NaN
7.png,Четки Мусульманские из агата 10 мм со вставкам...
8.png,Ювелирные мусульманские религиозные четки Куб...
9.png,Четки мусульманские изготовлены из граненого ч...
10.png,NaN
11.png,NaN
